# Usage Guide

In this sample piece, we will go through how to use the elliptical copulae as defined in the `copulae` package. We will not be covering what elliptical copula are. 

We will use the residuals data from the package for this tutorial. The data is a historical realization of the unknown processes. Each column follows their distinct (and unknown) process. However, these processes are related (have a dependency structure) to one another. Our task is to learn the dependency structure so that we could simulate future events.

This example is essentially a stripped down case of the GARCH-Copula model, which is common in certain industries.

In [1]:
from copulae.datasets import load_residuals

residuals = load_residuals()
residuals.head()

,A,B,C,D,E,F,G
0,0.730967,0.530860,0.287320,1.193049,0.019040,1.100507,0.278214
1,2.067853,-1.181313,-2.546173,0.381538,-0.038734,0.269874,-0.603940
2,-2.181835,0.380326,0.928632,-0.316861,0.106473,-0.324854,-0.447824
3,0.445040,0.734531,-0.133299,-0.374091,0.173616,-0.319402,-0.775106
4,0.296363,3.024053,0.815791,1.168521,0.134044,1.110424,1.705190


We will use both the `GaussianCopula` and the `StudentCopula`. But let's first start off with the Gaussian copula.

## Gaussian Copula

An alias of the `GaussianCopula` is `NormalCopula`. We can use either as they both refer to the same underlying structure.

In [2]:
from copulae import GaussianCopula

_, ndim = residuals.shape
g_cop = GaussianCopula(dim=ndim)  # initializing the copula
g_cop.fit(residuals)  # fit the copula to the data

Internally, the `fit` method will convert the data to pseudo observations so there is no need to for that sort of data treatment prior. However, even if your data is already in pseudo observations, there will be no change to the results as the transformation is monotonic in nature. 

To understand the quality of the fit, you can use the `summary` method.

In [3]:
g_cop.summary()

Gaussian Copula Summary
================================================================================
Gaussian Copula with 7 dimensions


Parameters
--------------------------------------------------------------------------------
 Correlation Matrix 
 1.000000  0.191081 -0.365944  0.128204  0.128853  0.110536  0.309972
 0.191081  1.000000  0.512683 -0.027041 -0.082239 -0.032021  0.207898
-0.365944  0.512683  1.000000  0.058283 -0.006468  0.055127  0.010648
 0.128204 -0.027041  0.058283  1.000000  0.624116  0.936115  0.590101
 0.128853 -0.082239 -0.006468  0.624116  1.000000  0.711072  0.416072
 0.110536 -0.032021  0.055127  0.936115  0.711072  1.000000  0.562437
 0.309972  0.207898  0.010648  0.590101  0.416072  0.562437  1.000000



                                  Fit Summary                                   
================================================================================
Log. Likelihood      : -810.9309775288916
Variance Estimate    : Not Implemented Yet
Method               : Maximum pseudo-likelihood
Data Points          : 394

Optimization Setup
--------------------------------------------------------------------------------
	bounds         : [(-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0)]
	options        : {'maxiter': 20000, 'ftol': 1e-06, 'iprint': 1, 'disp': False, 'eps': 1.5e-08}
	method         : SLSQP

Results
--------------------------------------------------------------------------------
	x              : [ 0.1910806  -0.36594407  0.12820358  0.1288529   0.11053562  0.30997244
  0.51268309 -0.02704128 -0.08223943 -0.03202084  0.20789778  0.0582833
 -0.0064678   0.05512652  0.01064777  0.62411582  0.93611501  0.59010119
  0.71107239  0.41607167  0.56243693]
	fun            : -810.9309775288916
	jac            : [-0.00166741 -0.00034864 -0.01106552  0.00116718  0.00198573  0.00135666
 -0.00127329  0.00070486 -0.00857957  0.00972401 -0.0005457   0.00277396
  0.00683637 -0.01141416 -0.00187204  0.02055458 -0.05338734 -0.01230849
 -0.02126702 -0.00407757  0.01490055]
	nit            : 26
	nfev           : 657
	njev           : 26
	status         : 0
	message        : Optimization terminated successfully.
	success        : True

### PDF, CDF and Random Variates

All the `pdf`, `cdf` and `random` methods of every copula work in the same manner. The only thing to note is that the input data for `pdf` and `cdf` must match the dimensions of the copula. In this case, we generate a 2x7 matrix, notice that the second dimension matches the dimension of the copula.

In [4]:
import numpy as np

random_matrix = np.random.uniform(0, 1, size=(2, 7))

pdf = g_cop.pdf(random_matrix)  # length 2 ndarray
cdf = g_cop.cdf(random_matrix)  # length 2 ndarray
rv = g_cop.random(2)  # shape is 2 by 7. 

### Reading Parameters

All copulas are parameterized in their own ways. Archimedeans, for example, is parameterized by a single $\theta$. For the Gaussian Copula, it is parameterized by the correlation matrix. To get the parameters for the copula, use the `params` property.

In [5]:
g_cop.params

array([ 0.1910806 , -0.36594407,  0.12820358,  0.1288529 ,  0.11053562,
        0.30997244,  0.51268309, -0.02704128, -0.08223943, -0.03202084,
        0.20789778,  0.0582833 , -0.0064678 ,  0.05512652,  0.01064777,
        0.62411582,  0.93611501,  0.59010119,  0.71107239,  0.41607167,
        0.56243693])

In this case, we get a vector instead of a correlation matrix (even though I mentioned that Gaussian Copulas are parameterized by the correlation matrix!). The answer is simple, these numbers are actually the diagonal elements of the correlation matrix. After all, in a correlation matrix, only the elements in the diagonals are "unique". 

For elliptical copulas, to see the correlation matrix, use the `sigma` property.

In [6]:
np.set_printoptions(linewidth=120)
g_cop.sigma

array([[ 1.        ,  0.1910806 , -0.36594407,  0.12820358,  0.1288529 ,  0.11053562,  0.30997244],
       [ 0.1910806 ,  1.        ,  0.51268309, -0.02704128, -0.08223943, -0.03202084,  0.20789778],
       [-0.36594407,  0.51268309,  1.        ,  0.0582833 , -0.0064678 ,  0.05512652,  0.01064777],
       [ 0.12820358, -0.02704128,  0.0582833 ,  1.        ,  0.62411582,  0.93611501,  0.59010119],
       [ 0.1288529 , -0.08223943, -0.0064678 ,  0.62411582,  1.        ,  0.71107239,  0.41607167],
       [ 0.11053562, -0.03202084,  0.05512652,  0.93611501,  0.71107239,  1.        ,  0.56243693],
       [ 0.30997244,  0.20789778,  0.01064777,  0.59010119,  0.41607167,  0.56243693,  1.        ]])

### Overwriting Parameters

The parameters are fit according to the empirical data. Many times, this is fine. However, there are instances where we want to overwrite the fitted parameters due to better understanding of the domain problem and any other reasons.

The basic way is to overwrite via the `params` property setter as seen in the example below.

```python
cop.params = 123
```

However, for the elliptical copulas, we have a convenience function that makes it easier to work with correlation matrix.

To overwrite single elements:

In [7]:
assert g_cop[0, 1] == g_cop[1, 0]
g_cop[0, 1] = 0.5
g_cop.sigma

array([[ 1.        ,  0.5       , -0.36594407,  0.12820358,  0.1288529 ,  0.11053562,  0.30997244],
       [ 0.5       ,  1.        ,  0.51268309, -0.02704128, -0.08223943, -0.03202084,  0.20789778],
       [-0.36594407,  0.51268309,  1.        ,  0.0582833 , -0.0064678 ,  0.05512652,  0.01064777],
       [ 0.12820358, -0.02704128,  0.0582833 ,  1.        ,  0.62411582,  0.93611501,  0.59010119],
       [ 0.1288529 , -0.08223943, -0.0064678 ,  0.62411582,  1.        ,  0.71107239,  0.41607167],
       [ 0.11053562, -0.03202084,  0.05512652,  0.93611501,  0.71107239,  1.        ,  0.56243693],
       [ 0.30997244,  0.20789778,  0.01064777,  0.59010119,  0.41607167,  0.56243693,  1.        ]])

To overwrite an entire correlation matrix, follow the code snippet below:

```python
my_corr_mat = # some correlation matrix
g_cop[:] = my_corr_mat  # this overwrites the entire correlation matrix
```

Behind the scenes, after overwriting the parameters, some transformations will be done to ensure that the resulting matrix remains positive semi-definite. If the matrix is already positive semi-definite, nothing will be done. But if it isn't, there will be some shifts to ensure that the resulting matrix has the minimum difference from the original matrix whilst being positive semi-definite. Thus don't be surprised if you change an element and notice that there are some bumps to the numbers.

## Student Copula

An alias of the `StudentCopula` is `TCopula`. We can use either as they both refer to the same underlying structure.

In [8]:
from copulae import StudentCopula

degrees_of_freedom = 5.5  # some random number, unnecessary to specify df but done for demonstration purposes
t_cop = StudentCopula(dim=ndim, df=degrees_of_freedom)
t_cop.fit(residuals)

The Student Copula differs from the Gaussian Copula in that it has one additonal parameter, the degrees of freedom `df`. This can be seen from the summary.

In [9]:
t_cop.summary()

Student Copula Summary
================================================================================
Student Copula with 7 dimensions


Parameters
--------------------------------------------------------------------------------
 Degree of Freedom  : 10.544548100216232

 Correlation Matrix 
 1.000000  0.177449 -0.374352  0.092292  0.111153  0.071978  0.265016
 0.177449  1.000000  0.525289 -0.055004 -0.077389 -0.065854  0.181235
-0.374352  0.525289  1.000000  0.057957  0.013242  0.060967  0.018375
 0.092292 -0.055004  0.057957  1.000000  0.630144  0.939848  0.579629
 0.111153 -0.077389  0.013242  0.630144  1.000000  0.716692  0.411570
 0.071978 -0.065854  0.060967  0.939848  0.716692  1.000000  0.558957
 0.265016  0.181235  0.018375  0.579629  0.411570  0.558957  1.000000



                                  Fit Summary                                   
================================================================================
Log. Likelihood      : -838.7958878924677
Variance Estimate    : Not Implemented Yet
Method               : Maximum pseudo-likelihood
Data Points          : 394

Optimization Setup
--------------------------------------------------------------------------------
	bounds         : [(0.0, inf), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0)]
	options        : {'maxiter': 20000, 'ftol': 1e-06, 'iprint': 1, 'disp': False, 'eps': 1.5e-08}
	method         : SLSQP

Results
--------------------------------------------------------------------------------
	x              : [10.5445481   0.17744906 -0.37435175  0.09229179  0.11115308  0.07197828  0.26501573  0.525289   -0.05500438
 -0.07738919 -0.06585391  0.18123546  0.05795655  0.01324215  0.06096713  0.01837462  0.63014357  0.93984772
  0.57962869  0.71669228  0.41157006  0.55895671]
	fun            : -838.7958878924677
	jac            : [ 0.00017432  0.00118992 -0.00216763  0.0095118   0.00413062 -0.01389253  0.00140214  0.00169014 -0.00523717
 -0.00253901  0.00823851 -0.00204636  0.00958759  0.00623762 -0.01572668  0.00230405  0.00212973  0.00072002
 -0.0117249  -0.0012354  -0.00203878  0.01228576]
	nit            : 32
	nfev           : 827
	njev           : 32
	status         : 0
	message        : Optimization terminated successfully.
	success        : True

In [10]:
t_cop.params

StudentParams(df=10.544548100216232, rho=array([ 0.17744906, -0.37435175,  0.09229179,  0.11115308,  0.07197828,  0.26501573,  0.525289  , -0.05500438,
       -0.07738919, -0.06585391,  0.18123546,  0.05795655,  0.01324215,  0.06096713,  0.01837462,  0.63014357,
        0.93984772,  0.57962869,  0.71669228,  0.41157006,  0.55895671]))

The rest of the `StudentCopula` work in the same way as the `GaussianCopula`. The only thing to note is that to change the degrees of freedom, you use `t_cop.df = 5`.

## Summary

That's all folks. We've gone through a simple example on 

1. How to fit a copula
2. How to get a summary of the fitted copula
3. How to get the PDF, CDF and Random Variates (these can be done even before fitting provided you set the parameters of the copula manually)
4. How to overwrite parameters of the copula

All the copulas pretty much follow a similar API so you probaby know about all of them already.